In [ ]:
from numcosmo_py import Ncm, Nc
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import warnings
from matplotlib.pyplot import cm

from scipy.integrate import odeint
from scipy.interpolate import interp1d
from numcosmo_py.plotting.tools import set_rc_params_article, latex_float

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [ ]:
set_rc_params_article(ncol=2)

In [ ]:
qgw = Nc.HICosmoQGW.new()
adiab = Nc.HIPertAdiab.new()

In [ ]:
qgw.props.xb = 1.0e35
qgw.props.w = 1.0e-5

In [ ]:

tAi = -200.0
tAf = -5.0e-2

ki = 1.0e0
kf = 1.0e2
k_a = np.geomspace(ki, kf, 1000)

adiab.set_k(1.0e-1)
adiab.set_ti(-200.0)
adiab.set_tf(60.0)
adiab.set_reltol(1.0e-11)
adiab.set_max_order_2(False)

Found, tAD = adiab.find_adiab_time_limit(qgw, tAi, tAf, 1.0e-3)

print(f"Found: {Found}, tAD: {tAD}")

In [ ]:
adiab.set_init_cond_adiab(qgw, tAD)
adiab.prepare(qgw)

In [ ]:
ta, ts = adiab.get_time_array()

In [ ]:
Pzeta = [adiab.eval_powspec_zeta_at(qgw, t) for t in ta]
PPsi = [adiab.eval_powspec_Psi_at(qgw, t) for t in ta]
Pdrho = [adiab.eval_powspec_drho_at(qgw, t) for t in ta]

In [ ]:
fig = plt.figure()

plt.plot(ta, Pzeta, label="$P_\zeta$")
plt.plot(ta, PPsi, label="$P_\Psi$")
plt.plot(ta, Pdrho, label="$P_{\delta_\rho}$")

plt.yscale('log')

plt.grid()
plt.legend()

In [ ]:
ka = np.geomspace(1.0e-1, 1.0e10, 1000)
tau_a = np.linspace(-250.0, 10.0, 200)

In [ ]:
adiab.prepare_spectrum(qgw, -1.0, 1.0e-7, ka, tau_a)

In [ ]:
Pzetak = adiab.eval_powspec_zeta(qgw)
PPsik = adiab.eval_powspec_Psi(qgw)
Pdrhok = adiab.eval_powspec_drho(qgw)

In [ ]:
fig = plt.figure()

tau_plot_a = [-180, -162.8, -10, 10]
for tau in tau_plot_a:
    plt.plot(ka, [Pzetak.eval(None, tau, k) for k in ka], label=f"$P_\zeta({tau})$")

plt.xscale('log')
plt.yscale('log')
plt.legend()
pass

In [ ]:
fig = plt.figure()

tau_plot_a = [-180, -162.8, -10, 10]
for tau in tau_plot_a:
    plt.plot(ka, [PPsik.eval(None, tau, k) for k in ka], label=rf"$P_\Psi({tau})$")

plt.xscale('log')
plt.yscale('log')
plt.legend()
pass

In [ ]:
fig = plt.figure()

tau_plot_a = [-180, -162.8, -10, 10]
for tau in tau_plot_a:
    plt.plot(ka, [Pdrhok.eval(None, tau, k) for k in ka], label=rf"$P_{{\delta_\rho}}({tau})$")

plt.xscale('log')
plt.yscale('log')
plt.legend()
pass

In [ ]:
psf_zeta = Ncm.PowspecFilter.new(Pzetak, Ncm.PowspecFilterType.TOPHAT)
psf_zeta.set_best_lnr0()

psf_Psi = Ncm.PowspecFilter.new(PPsik, Ncm.PowspecFilterType.TOPHAT)
psf_Psi.set_best_lnr0()

psf_drho = Ncm.PowspecFilter.new(Pdrhok, Ncm.PowspecFilterType.TOPHAT)
psf_drho.set_best_lnr0()


In [ ]:
psf_zeta.prepare(qgw)
psf_Psi.prepare(qgw)
psf_drho.prepare(qgw)

In [ ]:
print(psf_zeta.eval_sigma(10.0, 1.0))
print(psf_Psi.eval_sigma(10.0, 1.0))
print(psf_drho.eval_sigma(10.0, 1.0))

In [ ]:
r_a = np.geomspace(psf_zeta.get_r_min(), psf_zeta.get_r_max(), 1000)
sigma_r = [psf_drho.eval_sigma(-170.0, r) for r in r_a]

In [ ]:
fig = plt.figure()

plt.plot(r_a, sigma_r)
plt.xscale('log')
plt.yscale('log')